In [7]:
from datasets import load_dataset
from openai import OpenAI
from pinecone import Pinecone
import os
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain

In [5]:
os.environ["OPENAI_API_KEY"] = ""
pc = Pinecone(api_key = "")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [13]:
flan_t5 = HuggingFaceHub(repo_id= "google/flan-t5-base", 
                         model_kwargs = {"temperature": 1e-10})

template = """Question: {question} \n\n

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm = flan_t5)

In [21]:
llm_chain.invoke("How many bits are there in a Mega Byte?")

{'question': 'How many bits are there in a Mega Byte?', 'text': '256'}

In [20]:
llm_chain.run("How many bits are there in a Mega Byte?")

'256'

In [22]:
question = "Which NFL team won the Super Bowl in the 2010 season"
llm_chain.run(question)

'san francisco 49ers'

In [24]:
qs = [
    {"question": "Which NFL team won the Super Bowl in the 2010 season"},
    {"question": "If I am 6ft 3 inches, how tall am I in cms?"},
    {"question": "Who was the 12th person on the moon?"},
    {"question": "Who wrote the Indian constitution?"}
]

res = llm_chain.generate(qs)

In [25]:
res

LLMResult(generations=[[Generation(text='san francisco 49ers')], [Generation(text='84 cm')], [Generation(text='samuel harris')], [Generation(text='samuel harvard')]], llm_output=None, run=[RunInfo(run_id=UUID('8e3a97c2-fc59-4530-8380-3e622d93519f')), RunInfo(run_id=UUID('dac03c89-ff2d-4e1c-9056-3613a0d42b1c')), RunInfo(run_id=UUID('6df912db-178f-43f1-acae-16d2d761b5e7')), RunInfo(run_id=UUID('f72fc0fe-5267-4c09-9b67-7b652fa9cf09'))])

In [26]:
multi_template = """ Answer the following questions one at a time.
Questions: {questions}\n
Answer: 
"""

long_prompt = PromptTemplate(template = multi_template, input_variables= ["questions"])

llm_chain = LLMChain(prompt= long_prompt, llm = flan_t5)

questions = ("Which NFL team won the Super Bowl in the 2010 season"+
             "If I am 6ft 3 inches, how tall am I in cms?"+
             "Who was the 12th person on the moon?"+
             "Who wrote the Indian constitution?")

res = llm_chain.run(questions)

In [27]:
res

'Indian independence'

In [35]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
llm_chain = LLMChain(prompt = long_prompt, llm = llm)
res = llm_chain.run(questions)
res

'1. The Green Bay Packers won the Super Bowl in the 2010 season.\n2. If you are 6ft 3 inches tall, you are approximately 190.5 cm tall.\n3. The 12th person on the moon was Eugene Cernan.\n4. The Indian constitution was written by a committee chaired by Dr. B.R. Ambedkar.'

In [36]:
print(res)

1. The Green Bay Packers won the Super Bowl in the 2010 season.
2. If you are 6ft 3 inches tall, you are approximately 190.5 cm tall.
3. The 12th person on the moon was Eugene Cernan.
4. The Indian constitution was written by a committee chaired by Dr. B.R. Ambedkar.
